# QuickSort

#### We start with importing code we might use to do the benchmarking

In [10]:
import random
import numpy as np
import timeit
import copy
import pandas
import matplotlib
import matplotlib.pyplot as plt
import math
from tqdm import tqdm

#### Then we implement the QuickSort code

In [11]:
def partition(arr,l,h): 
    """
    Finding the pivot
    """
    i = ( l - 1 ) 
    x = arr[h] 
    for j in range(l , h): 
        if arr[j] <= x: 
            # increment index of smaller element 
            i = i+1
            arr[i],arr[j] = arr[j],arr[i] 

    arr[i+1],arr[h] = arr[h],arr[i+1] 
    return (i+1) 
  
# Function to do Quick sort 
# arr[] --> Array to be sorted, 
# l  --> Starting index, 
# h  --> Ending index 
def quickSortIterative(arr,l,h): 
    """
    Iterative implementation of quick sort
    """
    # Create an auxiliary stack 
    size = h - l + 1
    stack = [0] * (size) 
  
    # initialize top of stack 
    top = -1
  
    # push initial values of l and h to stack 
    top = top + 1
    stack[top] = l 
    top = top + 1
    stack[top] = h 
  
    # Keep popping from stack while is not empty 
    while top >= 0: 
  
        # Pop h and l 
        h = stack[top] 
        top = top - 1
        l = stack[top] 
        top = top - 1
  
        # Set pivot element at its correct position in 
        # sorted array 
        p = partition( arr, l, h ) 
  
        # If there are elements on left side of pivot, 
        # then push left side to stack 
        if p-1 > l: 
            top = top + 1
            stack[top] = l 
            top = top + 1
            stack[top] = p - 1
  
        # If there are elements on right side of pivot, 
        # then push right side to stack 
        if p+1 < h: 
            top = top + 1
            stack[top] = p + 1
            top = top + 1
            stack[top] = h 

def quicksort(data):
    """
    Implementation of quick sort which immediatly calls a helper function
    """
    quickSortIterative(data, 0, len(data)-1)

#### Then we make a function that test if our implement of the code i working as it should

In [12]:
def test_sorting_algorithm(algorithm): 
    """
    Function to test the correctness of a sorting algorithm
    Generating numpy array with random integers to be tested on
    Tests it 10 times and then i assume that it is correct
    """
    for i in range(1000):
        A = np.random.randint(1000, size=100)
        A_copy = A.copy()
        algorithm(A_copy)      
        assert A_copy.tolist() == sorted(A), 'The implementation of %s is wrong'% (algorithm.__name__)

In [13]:
test_sorting_algorithm(quicksort)

#### In the benchmarking we will need random generated data, so we will make som functions that gives us diffrent kind of random genreated data. This will test the sort funciton for best, worst and avrage case

In [14]:
def ascending_list_int(n):
    """
    Returns a ascending list with values from 0 to n with length n
    """
    List = [i for i in range(n)]
    return List

def descending_list_int(n):
    """
    Returns a descending list with values from n to 0 with length n
    """
    List = [i for i in range(n - 1, -1, -1)]
    return List

def random_list_int(n):
    """
    Returns a list of random integers from -n to n with length n
    """
    List = [random.randint(-n, n) for _ in range(n)]
    return List
  
def random_list_float(n):
    """
    Returns a list of length n with random float values from -n to n
    """
    List = [random.uniform(-n, n) for _ in range(n)]
    return List


def random_charlist(n):
    """
    Returns a list of length n with random characters
    """
    List = [random.choice('abcdefghisjklmnopqrstuvwxyz') 
                 for _ in range(n)]
    
    return List

test_data_list = [ascending_list_int, descending_list_int, random_list_int, random_list_float, random_charlist]

#### Making the time function that will time the sort function for the benchmarking

In [15]:
def time_function(sort_function, test_data):
    """
    Actual function which does the timing
    """
    clock = timeit.Timer('func(copy(data))',
                       globals={'func': sort_function, 'data': test_data, 
                                'copy': copy.copy})
    

    n_ar, t_ar = clock.autorange()
    
    data = np.array(clock.repeat(repeat=7, number=n_ar)) / n_ar
    
    sort = pandas.DataFrame(data)
    sort.to_pickle("quick_sort_times")
    
    return np.min(data)

#### Function that does the benchmarking

In [16]:
test_sizes = [10, 100, 1000, 10000, 100000]
test_size2 = [10, 100, 1000]

def benchmark_function(sort_function):
    data1 = [[], []]
    data2 = [[], []]
    data3 = [[], []]
    data4 = [[], []]
    data5 = [[], []]
   
    for size in tqdm(test_sizes):
        data3[0].append(size) 
        data4[0].append(size)
        data5[0].append(size)
        data3[1].append(time_function(sort_function,random_list_int(size)))
        data4[1].append(time_function(sort_function,random_list_float(size)))
        data5[1].append(time_function(sort_function,random_charlist(size)))
    
    for size in tqdm(test_size2):
        data1[0].append(size) 
        data2[0].append(size)
        data1[1].append(time_function(sort_function,ascending_list_int(size)))
        data2[1].append(time_function(sort_function,descending_list_int(size)))
       
   
    all_data = [data1, data2, data3, data4, data5]
    
    quick_sort = pandas.DataFrame(all_data)
    
    quick_sort.to_pickle("quick_sort")
    
    return all_data

benchmark_quick = benchmark_function(quicksort)


100%|██████████| 3/3 [00:16<00:00,  5.46s/it]